In [1]:
import numpy as np
import pandas as pd
from scipy import optimize

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# directory from your google drive to include data and other excel files.
import os
os.chdir('/content/drive/My Drive/AOP_Final_Project')

In [4]:
dataframe_fwd_3320_T_11 = pd.read_excel("fwd_3320_T_11.xlsx")

In [5]:
type(dataframe_fwd_3320_T_11.MarketVol)

pandas.core.series.Series

In [6]:
#Implementation of Hagan's formula eq2.17 for beta = 1
def SABR2Black76(F0, K, T, alpha, rho, nu):
  z = nu / alpha * np.log(F0 / K)
  xz = np.log((np.sqrt(1 - 2*rho*z + z**2) + z - rho) / (1 - rho))
  if z==0:
    return alpha*(1+ (0.25*rho*alpha*nu + ((2-3*(rho**2))/24)*(nu**2))*T)
  else:
    return alpha*(z/xz)*(1+ (0.25*rho*alpha*nu + ((2-3*(rho**2))/24)*(nu**2))*T)

In [7]:
#test case
alpha0 = 0.05
rho0 = 0.05
nu0 = 0.7
TargetVal = np.zeros(len(dataframe_fwd_3320_T_11.MarketVol))
for i in range(len(dataframe_fwd_3320_T_11.MarketVol)):
  TargetVal[i] = SABR2Black76(3320.336332, dataframe_fwd_3320_T_11.Strike[i], 0.92, alpha0, rho0, nu0)

TargetVal

array([0.54886745, 0.46225712, 0.41003265, 0.37211246, 0.3421141 ,
       0.31716585, 0.2957232 , 0.27685757, 0.25996606, 0.24463446,
       0.23056552, 0.2175384 , 0.20538431, 0.19397113, 0.18319337,
       0.17296527, 0.16321605, 0.15388653, 0.14492663, 0.1362936 ,
       0.1279507 , 0.11986636, 0.11201369, 0.1043705 , 0.09692   ,
       0.08965268, 0.08257056, 0.07569612, 0.06909194, 0.06290458,
       0.05746065, 0.05343278, 0.05184378, 0.05322538, 0.05681587,
       0.06149075, 0.06655999, 0.07170317, 0.07678147, 0.08173648,
       0.08654572, 0.09120318, 0.0957105 , 0.10007287, 0.10429699,
       0.10839003, 0.11235914, 0.11621122, 0.11995278, 0.1235899 ,
       0.12712826])

In [8]:
# objective function
def objective(params, *args):
    alpha, rho, nu = params
    F, K, T, market_vols = args # constant arguments
    model_vols = np.zeros(len(dataframe_fwd_3320_T_11.MarketVol))
    for i in range(len(dataframe_fwd_3320_T_11.MarketVol)):
      model_vols[i] = SABR2Black76(F, K[i], T, alpha, rho, nu) # from Hagan's formula eq2.17
    return np.sum((model_vols - market_vols)**2) # SSE

In [9]:
params_init = np.array([alpha0, rho0, nu0]) # initila parameters **choice matters**
F = 3320.336332
T = 0.92
K = dataframe_fwd_3320_T_11.Strike
market_vols = dataframe_fwd_3320_T_11.MarketVol
args = (F, K, T, market_vols)

In [10]:
# in-built optimization from scipy
result = optimize.minimize(objective, params_init, args=args, method='Nelder-Mead')

In [11]:
alpha_opt, rho_opt, nu_opt = result.x
print("Optimized parameters [alpha, rho, nu]: ", alpha_opt, rho_opt, nu_opt)

Optimized parameters [alpha, rho, nu]:  0.1446698927514169 -0.6312331064985108 0.7908025105144357


In [12]:
def objective(params, F, K, T, market_vols):
    alpha, rho, nu = params
    model_vols = np.zeros(len(market_vols))
    for i in range(len(market_vols)):
        model_vols[i] = SABR2Black76(F, K[i], T, alpha, rho, nu)
    return np.sum((model_vols - market_vols)**2)

# Define a simple numerical approximation for the gradient
def gradient(params, F, K, T, market_vols, eps=1e-4):
    grad = np.zeros(len(params))
    for i in range(len(params)):
        params_plus_eps = np.copy(params)
        params_plus_eps[i] += eps
        grad[i] = (objective(params_plus_eps, F, K, T, market_vols) - objective(params, F, K, T, market_vols)) / eps
    return grad

# Initialize the parameters
params = np.array([alpha0, rho0, nu0])

# Initialize the learning rate
learning_rate = 0.01

# Number of iterations for the gradient descent
num_iterations = 100000

# Set the market data
F = 3320.336332
T = 0.92
K = dataframe_fwd_3320_T_11.Strike.values
market_vols = dataframe_fwd_3320_T_11.MarketVol.values

# Stochastic Gradient Descent
for i in range(num_iterations):
    grad = gradient(params, F, K, T, market_vols)
    params -= learning_rate * grad

print("Optimized parameters [alpha, rho, nu]: ", params)

Optimized parameters [alpha, rho, nu]:  [ 0.14453328 -0.6309153   0.79122673]
